<a href="https://colab.research.google.com/github/shubhangkhare/Learnings/blob/main/UP20%20Analytics%20Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import Libraries

In [1]:
!pip install mplfinance yfinance -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.7 MB/s eta 0:00:00


In [10]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

# Load NSE Tickers

In [11]:
tickers = pd.read_html('https://ournifty.com/stock-list-in-nse-fo-futures-and-options.html#:~:text=NSE%20F%26O%20Stock%20List%3A%20%20%20%20SL,%20%201000%20%2052%20more%20rows%20')[0]

In [13]:
tickers.head()

,SL NO,F&O STOCKS NAME,SYMBOL,LOT SIZE
0,1,NIFTY MID SELECT,MIDCPNIFTY,75
1,2,NIFTY FINANCIAL SERVICES,FINNIFTY,40
2,3,NIFTY BANK,BANKNIFTY,15
3,4,NIFTY 50,NIFTY,25
4,5,AARTI INDUSTRIES LTD,AARTIIND,1000


# Plot Candlesticks

In [16]:
# Retrieve AAPL historical data
symbol = "ADANIPOWER" + ".NS" # Share symbol + NSE suffix
ticker = yf.Ticker(symbol)
data = ticker.history(period="1mo")

# Create candlestick chart
fig = go.Figure(data=[go.Candlestick(x=data.index,
                                     open=data['Open'],
                                     high=data['High'],
                                     low=data['Low'],
                                     close=data['Close'])])

# Customize the chart layout
fig.update_layout(title=f"{symbol} Candlestick Chart (1 Month)",
                  yaxis_title="Price",
                  xaxis_rangeslider_visible=False)

# Display the chart
fig.show()

# UP 20 - Code

In [17]:
data.reset_index(inplace = True)
data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-10-07 00:00:00+05:30,641.450012,645.950012,613.400024,625.450012,2859604,0.0,0.0
1,2024-10-08 00:00:00+05:30,624.900024,645.900024,613.200012,640.599976,2350816,0.0,0.0
2,2024-10-09 00:00:00+05:30,646.400024,649.000000,631.549988,634.299988,1270367,0.0,0.0
3,2024-10-10 00:00:00+05:30,641.950012,648.400024,631.049988,645.400024,1649770,0.0,0.0
4,2024-10-11 00:00:00+05:30,645.000000,645.400024,637.450012,642.299988,1109827,0.0,0.0


In [4]:
data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-10-07 00:00:00-04:00,224.500000,225.690002,221.330002,221.690002,39505400,0.0,0.0
2024-10-08 00:00:00-04:00,224.300003,225.979996,223.250000,225.770004,31855700,0.0,0.0
2024-10-09 00:00:00-04:00,225.229996,229.750000,224.830002,229.539993,33591100,0.0,0.0
2024-10-10 00:00:00-04:00,227.779999,229.500000,227.169998,229.039993,28183500,0.0,0.0
2024-10-11 00:00:00-04:00,229.300003,229.410004,227.339996,227.550003,31759200,0.0,0.0
